## Setup

In [ ]:
!pip install dtaidistance
!pip install tensorflow==2.15.0
!pip install tensorflow_probability==0.23.0

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
import os
from dtaidistance import dtw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Configuration

In [ ]:
halt_at_end = True
save_model = False

In [ ]:
t_days = 5 # Lead time (in days)
base_carrington = 27.26

## Initialization

In [ ]:
!rm -rf ch_data
!mkdir ch_data

In [ ]:
!unzip 2012.zip -d ch_data
!unzip 2013.zip -d ch_data
!unzip 2014.zip -d ch_data
!unzip 2015.zip -d ch_data
!unzip 2016.zip -d ch_data
!unzip 2017.zip -d ch_data
!unzip 2018.zip -d ch_data
!unzip 2019.zip -d ch_data
!unzip 2020.zip -d ch_data
!unzip 2021.zip -d ch_data
!unzip 2022.zip -d ch_data

In [ ]:
from textwrap import fill
path = 'ch_data/features/ch_measurements'
buffer_size = 4
data = []
for file in os.listdir(path):
  try:
    d = file.split('.')
    d = d[0]+'-'+d[1]+'-'+d[2]
    feats = pd.read_csv(os.path.join(path, file))
    cols = feats.columns.values
    cols[0] = 'ID'
    feats.columns = cols
    feats = feats.drop(columns=['ID'])
    cols = feats.columns
    n_holes = buffer_size
    new_cols = np.stack([cols for i in range(n_holes)])
    for i in range(len(new_cols)):
      for c in range(len(new_cols[i])):
        new_cols[i][c] = new_cols[i][c]+'_'+str(i)
    new_cols = new_cols.flatten()
    vals = np.concatenate([feats.values.flatten(), [0 for _ in range(n_holes*len(cols)-len(feats.values.flatten()))]]).reshape((1, n_holes*len(cols)))
    final_feats = pd.DataFrame(vals, columns=new_cols, index=[pd.to_datetime(d)])
    data.append(final_feats)
  except:
    continue

ch_data = pd.concat(data).sort_index()
new_idx = pd.date_range(ch_data.index[0], ch_data.index[-1], freq='1d')
ch_data = ch_data.reindex(new_idx, fill_value=0)
new_idx = pd.date_range(ch_data.index[0], ch_data.index[-1], freq='1h')
ch_data = ch_data.reindex(new_idx)
drop_list = []
for c in ch_data.columns:
  if not('area' in c) and not('top' in c) and not('bot' in c) and not('left' in c) and not('right' in c) and not('flux' in c):
    drop_list.append(c)
  if 'flux' in c:
    ch_data[c] = ch_data[c].apply(np.abs)
ch_data=ch_data.drop(columns=drop_list)
for c in ch_data.columns:
  ch_data[c] = ch_data[c].astype(np.float32)
ch_data = ch_data.interpolate(method='time')
ch_data

In [ ]:
wind_data_path = 'drive/MyDrive/SpaceWeatherData/ACE/omni_1hr_clean.csv'
wind_data = pd.read_csv(wind_data_path, parse_dates=[0], index_col=0).interpolate(method='time')
wind_data['B'] = np.sqrt(wind_data['BR']**2+wind_data['BT']**2+wind_data['BN']**2)
wind_data['T'] = np.log10(wind_data['T'])
wind_data['N'] = np.log10(wind_data['N'])
wind_data

In [ ]:
wind_data = ch_data.join(wind_data)
wind_data = wind_data.join(wind_data.shift(round(24*(base_carrington-t_days))), rsuffix='_Car').dropna(how='any')
wind_data

In [ ]:
in_cols = ['V', 'N', 'T', 'B', 'V_Car', 'N_Car', 'T_Car', 'B_Car']+ch_data.columns.tolist()+(ch_data.columns.values+'_Car').tolist()
out_cols = ['V']

In [ ]:
in_mu = wind_data[in_cols].mean().values
in_std = wind_data[in_cols].std().values
out_mu = wind_data[out_cols].mean().values
out_std = wind_data[out_cols].std().values

In [ ]:
def standardize(x, inpt=True):
  if inpt:
    return (x-in_mu)/in_std
  else:
    return (x-out_mu)/out_std

def destandardize(x, inpt=True):
  if inpt:
    return x*in_std+in_mu
  else:
    return x*out_std+out_mu

In [ ]:
in_size= round(24*5)
offset = round(24*t_days)
batch_size = 1024

## Dataset generation

In [ ]:
idx = wind_data.index.values
l = len(idx)

train_p = 0.7
train_portion = idx[:round(train_p*l)]
val_portion = idx[round(train_p*l):round(((1-train_p)/2+train_p)*l)]
test_portion = idx[round(((1-train_p)/2+train_p)*l):]

In [ ]:
display(train_portion)
display(len(train_portion))
print()
display(val_portion)
display(len(val_portion))
print()
display(test_portion)
display(len(test_portion))

In [ ]:
train_set = keras.utils.timeseries_dataset_from_array(
    standardize(wind_data.loc[train_portion[0]:train_portion[-1], in_cols][:-offset]),
    standardize(wind_data.loc[train_portion[0]:train_portion[-1], out_cols][offset:], inpt=False),
    start_index=offset, sequence_length=in_size, batch_size=batch_size, shuffle=True)

val_set = keras.utils.timeseries_dataset_from_array(
    standardize(wind_data.loc[val_portion[0]:val_portion[-1], in_cols][:-offset]),
    standardize(wind_data.loc[val_portion[0]:val_portion[-1], out_cols][offset:], inpt=False),
    start_index=offset, sequence_length=in_size, batch_size=batch_size)

test_set = keras.utils.timeseries_dataset_from_array(
    standardize(wind_data.loc[test_portion[0]:test_portion[-1], in_cols][:-offset]),
    standardize(wind_data.loc[test_portion[0]:test_portion[-1], out_cols][offset:], inpt=False),
    start_index=offset, sequence_length=in_size, batch_size=batch_size)

## Model definition

In [ ]:
use_exact_kl = False

def prior(kernel_size, bias_size, dtype=None):
    delimiters = [400, 600, 700, 800]
    prop = len(wind_data[wind_data.V < delimiters[0]])/len(wind_data)
    props = [prop]
    pstds = [standardize(wind_data[wind_data.V < delimiters[0]][in_cols].std(), inpt=False)[0]]
    pmus = [standardize(wind_data[wind_data.V < delimiters[0]][in_cols].mean(), inpt=False)[0]]
    for i in range(1, len(delimiters)-1):
      d, dp = delimiters[i], delimiters[i+1]
      aux = wind_data[wind_data.V >= d]
      aux = aux[aux < dp]
      props.append(len(aux)/len(wind_data))
      pstds.append(standardize(aux[in_cols].std(), inpt=False)[0])
      pmus.append(standardize(aux[in_cols].mean(), inpt=False)[0])
      if np.isnan(pstds[-1]):
        pstds[-1] = pstds[-2]
      if np.isnan(pmus[-1]):
        pmus[-1] = pmus[-2]
    props.append(len(wind_data[wind_data.V >= delimiters[-1]])/len(wind_data))
    pstds.append(standardize(wind_data[wind_data.V >= delimiters[-1]][in_cols].std(), inpt=False)[0])
    pmus.append(standardize(wind_data[wind_data.V >= delimiters[-1]][in_cols].mean(), inpt=False)[0])
    if np.isnan(pstds[-1]):
        pstds[-1] = pstds[-2]
    if np.isnan(pmus[-1]):
      pmus[-1] = pmus[-2]
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t:
                tfp.distributions.Mixture(
                    cat=tfp.distributions.Categorical(probs=props),
                    components=[
                    tfp.distributions.MultivariateNormalDiag(
                        loc=tf.zeros(n)+pmus[i], scale_diag=tf.ones(n)*pstds[i]
                    ) for i in range(len(props))
                    ]
                )
            )
        ]
    )
    return prior_model

def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

def negative_loglikelihood(targets, estimated_distribution):
  return -estimated_distribution.log_prob(targets)

In [ ]:
hidden_units = [60]

def build_model(train_size):
  x=in_layer = keras.layers.Input((in_size, len(in_cols)))
  x = keras.layers.LSTM(len(in_cols), return_sequences=False, activation='tanh')(x)
  features=x
  for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation= "tanh",
            kl_use_exact = use_exact_kl
        )(features)
  distribution_params = keras.layers.Dense(units=2)(features)
  outputs = tfp.layers.IndependentNormal(1)(distribution_params)

  model = keras.Model(inputs=in_layer, outputs=outputs)
  return model

## Model fitting

In [ ]:
model = build_model(len(train_set))

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(loss=negative_loglikelihood,
              optimizer=keras.optimizers.RMSprop(clipnorm=1.0),
              metrics=['mse'])

In [ ]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, restore_best_weights=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.TerminateOnNaN()
]

In [ ]:
epochs = 3000

In [ ]:
model.fit(train_set, validation_data = val_set, epochs=epochs,
          callbacks=callbacks)

## Model evaluation

In [ ]:
model.evaluate(test_set)

In [ ]:
t = out_cols[0]
units = {'V': '$\mathrm{km}\,\mathrm{s}^{-1}$',
         'T': 'K',
         'N': 'cm$^{-3}$',
         'B': 'nT',
         'BR': 'nT',
         'BN': 'nT',
         'BT': 'nT'}

t_names = {
    'V': 'V',
    'T': 'T',
    'N': '\\rho',
    'B': '|\mathbf{B}|',
    'BR': '-B_x',
    'BT': 'B_y',
    'BN': 'B_z'
}

In [ ]:
def mse(a, b):
  return np.mean((a-b)**2)

def mae(a, b):
  return np.mean(np.abs(a-b))

def rmse(a, b):
  return np.sqrt(mse(a, b))

def cc(a, b):
  return np.cov(a, b)[1, 0]/(np.std(a)*np.std(b))

def r2(a, b):
  return 1-np.sum((a-b)**2)/np.sum((a-np.mean(a))**2)

def evaluate():
  print(f'Number of timesteps analyzed = {len(all_means)}')
  print(f'Maximum width of 95% CI = {np.max(np.abs(upper-lower)):.2f} {units[t]}')
  print(f'Average width of 95% CI = {np.mean(np.abs(upper-lower)):.2f} {units[t]}')
  print(f'Maximum punctual error magnitude for mean = {np.max(np.abs(all_truths-all_means)):.2f} {units[t]}')
  print(f'Maximum punctual error magnitude for upper bound = {np.max(np.abs(all_truths-upper)):.2f} {units[t]}')
  print(f'Maximum punctual error magnitude for lower bound = {np.max(np.abs(all_truths-lower)):.2f} {units[t]}')
  acc = np.sum(all_truths[all_truths >= lower] <= upper[all_truths >= lower])/len(all_truths)
  print(f'95% CI interval accuracy = {acc:.2%}')
  print()

  print('Evaluation for mean value:')
  print(f'\tRMSE = {rmse(all_truths, all_means):.2f} {units[t]}')
  print(f'\tMAE = {mae(all_truths, all_means):.2f} {units[t]}')
  print(f'\tCC = {cc(all_truths, all_means):.2%}')
  print(f'\tR2 = {r2(all_truths, all_means):.2%}')
  print(f'\tDTW = {dtw.distance_fast(all_truths, all_means):.2f}')
  print()
  print('Evaluation for upper bound:')
  print(f'\tRMSE = {rmse(all_truths, upper):.2f} {units[t]}')
  print(f'\tMAE = {mae(all_truths, upper):.2f} {units[t]}')
  print(f'\tCC = {cc(all_truths, upper):.2%}')
  print(f'\tR2 = {r2(all_truths, upper):.2%}')
  print(f'\tDTW = {dtw.distance_fast(all_truths, upper):.2f}')
  print()
  print('Evaluation for lower bound:')
  print(f'\tRMSE = {rmse(all_truths, lower):.2f} {units[t]}')
  print(f'\tMAE = {mae(all_truths, lower):.2f} {units[t]}')
  print(f'\tCC = {cc(all_truths, lower):.2%}')
  print(f'\tR2 = {r2(all_truths, lower):.2%}')
  print(f'\tDTW = {dtw.distance_fast(all_truths, lower):.2f}')

In [ ]:
x_axis = wind_data.loc[test_portion[0]:test_portion[-1]][offset:].index

In [ ]:
# Running in single execution mode for quick debugging.
# Although performance metrics will resemble those obtained in proper sampling (N >= 10),
# they will be slightly perturbed on account of statistical noise.
# For proper runs, encapsulate in an external for loop guided by N, then compute the
# corresponding temporal aggregations (mean, epistemic uncertainty and aleatoric uncertainty).
all_means = []
all_stds = []
all_truths = []
for batch in tqdm(test_set):
  X, y = batch
  all_truths += y.numpy().flatten().tolist()
  prediction_distribution = model(X)
  prediction_mean = prediction_distribution.mean().numpy().flatten().tolist()
  all_means += prediction_mean
  prediction_stdv = prediction_distribution.stddev().numpy().flatten()
  all_stds += prediction_stdv.tolist()

In [ ]:
all_means=np.array(all_means).flatten()
all_stds = np.array(all_stds).flatten()
all_truths = destandardize(np.array(all_truths).flatten(), inpt=False)

upper = destandardize((all_means + (1.96 * all_stds)).tolist(), inpt=False)
lower = destandardize((all_means - (1.96 * all_stds)).tolist(), inpt=False)
all_means= destandardize(all_means, inpt=False)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(x_axis[len(x_axis)-len(all_truths):], all_truths, label='Observation')
plt.plot(x_axis[len(x_axis)-len(all_truths):], all_means, linestyle='--', label='Predicted $\\mu$')
plt.fill_between(x_axis[len(x_axis)-len(all_truths):], upper, lower, alpha=.25,
                 color='orange', label='95% CI')
plt.ylabel(f'${t_names[t]}$ ({units[t]})')
plt.legend()
plt.show()
plt.close()

In [ ]:
evaluate()

## Kernel halting

Uncomment `halt` (undefined function) for planned attended usage; comment it for safe, resource-effective unattended usage.

In [ ]:
if save_model:
  from google.colab import files
  model.save_weights(f'solar_wind_{t}_{t_days}_model_for_plots.h5', overwrite=True) # save only the weights because Keras has issues saving full variational layers (Bayesian inference parameters are saved)
  files.download(f'solar_wind_{t}_{t_days}_model_for_plots.h5')

In [ ]:
if halt_at_end:
  halt

In [ ]:
# Disconnect to prevent resource misuse
from google.colab import runtime
runtime.unassign()